In [1]:
! git clone https://github.com/vhiwase/LMBERT.git

Cloning into 'LMBERT'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [1]:
%cd LMBERT

/content/LMBERT


In [2]:
! ls

dataset.csv   LICENSE		README.md	  SiamesePreTrainer.py
inference.py  LMBERTDataset.py	requirements.txt
__init__.py   LMBERT.py		run.py


In [3]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from LMBERT import LMBERT
from SiamesePreTrainer import SiamesePreTrainer
from LMBERTDataset import LMBERTDataset
import pandas as pd

In [17]:
import torch
if torch.cuda.is_available():
     device = 'cuda'
else:
     device = 'cpu'
print('device:', device)

device: cuda


In [18]:
model = LMBERT('bert-base-uncased', device=device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [19]:
df = pd.read_csv('dataset.csv')
base_language_sentences = df['translation'].tolist()
target_language_sentences = df['transliteration'].tolist()
batch_size = 16

In [20]:
splitter = lambda percentage: int((len(base_language_sentences)*percentage//batch_size)*batch_size)
training_splitter = splitter(.8)
print(f"training_splitter: {training_splitter}")

training_splitter: 118176


In [21]:
train_dataset = LMBERTDataset(base_language_sentences[:training_splitter], target_language_sentences[:training_splitter])
eval_dataset = LMBERTDataset(base_language_sentences[training_splitter:], target_language_sentences[training_splitter:])


  0%|          | 0/118176 [00:00<?, ?it/s]

  0%|          | 0/118176 [00:00<?, ?it/s]

  0%|          | 0/29555 [00:00<?, ?it/s]

  0%|          | 0/29555 [00:00<?, ?it/s]

In [22]:
! rm -rf /content/LMBERT/LMBERT
trainer = SiamesePreTrainer(
            model=model, 
            train_dataset=train_dataset, 
            eval_dataset=train_dataset,
            epochs=10,
            batch_size=batch_size,
            device=device) 

In [ ]:
trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/7386 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from transformers import pipeline

In [ ]:
LMBERT_fill_mask  = pipeline("fill-mask", model="LMBERT/best-model-3.4319419860839844", tokenizer="LMBERT/best-model-3.4319419860839844")
text = " mera naam [MASK] hai"
results = LMBERT_fill_mask(text)
results

In [ ]:
LMBERT_fill_mask  = pipeline("text-generation", model="LMBERT/best-model-3.4319419860839844", tokenizer="LMBERT/best-model-3.4319419860839844")
text = " mera naam Vaibhav hai"
results = LMBERT_fill_mask(text)
results